In [112]:
import websocket
import json
from urllib.parse import quote

In [113]:
ws = websocket.WebSocket()
website_id='google.com'
ws.connect(f'wss://plankton-app-5rfza.ondigitalocean.app/websites?scraping={website_id}')
ws.recv()

'{"id": "google.com", "scraping": true}'

In [114]:
ws.close()

In [151]:
ws = websocket.WebSocket()
page_id = 'https://betway.com/es/sports/evt/10647401'
website_id='betway.com'
# uri = 'wss://plankton-app-5rfza.ondigitalocean.app'
uri='ws://localhost:8000'
ws.connect(f'{uri}/games/{website_id}?game_id={page_id}')
ws.recv()

WebSocketConnectionClosedException: Connection to remote host was lost.

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_recv',
 '_send',
 'abort',
 'close',
 'connect',
 'connected',
 'cont_frame',
 'fileno',
 'frame_buffer',
 'get_mask_key',
 'getheaders',
 'getstatus',
 'getsubprotocol',
 'gettimeout',
 'handshake_response',
 'headers',
 'is_ssl',
 'lock',
 'next',
 'ping',
 'pong',
 'readlock',
 'recv',
 'recv_data',
 'recv_data_frame',
 'recv_frame',
 'send',
 'send_binary',
 'send_close',
 'send_frame',
 'set_mask_key',
 'settimeout',
 'shutdown',
 'sock',
 'sock_opt',
 'status',
 'subprotocol',
 'timeout']

In [72]:
from core.models import r
from redis.commands.search.query import Query, NumericFilter

In [148]:
r.ft('idxGames').search(Query('@scraping:{true}').paging(0, 5))

Result{0 total, docs: []}

In [104]:
from core.webdriver import init_driver
driver = init_driver(False)

In [105]:
driver.execute_script("window.open('');")